# Open Data University - Accidentologie à vélo

Dans le cadre de l'opportunité Open Data University offert par *Latitudes*, nous nous sommes proposés pour un projet de *data visualization* autour de la problematiques de l'accidentologie à vélo pour apporté notre pierre à l'édifice parmis les grandes restructuration de la mobilité en France.  

## SPRINT 01 - DATA CLEANING

Qui dit *data visualization* dit *data*.  
Nous avons donc trouver après recherche sur *www.data.gouv.fr* [les bases de données annuelles des accidents corporels de la circulation routière](https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/), un ensemble de données qui donne notamment les accidents en vélo et en VAE (Vélo à Assistance Electrique). Cependant le dataset couvre un scope plus grand que notre sujet puisqu'il se porte sur l'intégralité des accidents de la circulation routière. Il nous faut donc tout d'abord procéder à un nettoyage de la donnée.

A partir de 2005, la notation des accidents est séparée entre 4 fichiers: Caractéristiques, Lieux, Véhicules, Usagers.  
Et C'est à partir du fichier Véhicules que nous allons donc pouvoir sélectionner les accidents impliquant seulement des vélo ou VAE.

In [17]:

pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
# import pandas to download, read and filter .csv files
import pandas as pd

In [19]:
# the filepath to the .json file that contains all the dataset urls
url_fp: str = "data/metadata/urls.json"

# convert metadata file to dataframe which
url_df: pd.DataFrame = pd.read_json(url_fp)

On télécharge depuis les urls le fichier qui donne les caractéristiques des véhicules dans les accidents.

In [20]:
valid_categories: list[str] = ['caractéristiques', 'lieux', 'véhicules', 'usagers']
valid_years: list[int]= [2017, 2018, 2019, 2020, 2021]

def get_base_url(year: int, category: str) -> str | None:
    if year not in valid_years or category not in valid_categories:
        return
    
    # Get year idx as year are in descendant order in metadata file
    year_idx: int = 2021 - year
    return url_df.at[year_idx, category]

v_url: str | None = get_base_url(2021, 'véhicules')
if v_url == None:
    exit(0)

"""
    Only use those columns because
    we only want to extract the accident id from the vehicule category
"""
v_col: list[str] = ['Num_Acc', 'catv']

# convert .csv file to pandas dataframe
v_df: pd.DataFrame = pd.read_csv(v_url, sep=';', usecols=v_col, on_bad_lines='skip')

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

: 

Puisque le fichier Véhicules donne pour chaque accident un numéro de categorie de véhicule (*catv*), nous pouvons donc extraire depuis ce fichier les numéro d'accident unique des véhicules qui nous concernent seulement.  
  
En l'occurence seuls deux catégories référencés dans le descriptif nous concernent: les bicyclettes (01) et les VAE (80).

In [ ]:
labels: list[str] = [1, 80]

# filter the dataframe to bycicles only
v_df.query("catv == @labels", inplace=True)

Nous épurons enfin le résultat en retirant du dataset les colonnes nous ayant permis de faire le filtre précédent et en supprimant les duplicatas (un accident pouvant impliquer plus d'un véhicules).

In [ ]:
# and then remove all columns but the accident id column
v_df = v_df.filter(items=['Num_Acc'])
# remove the duplicates
v_df.drop_duplicates(inplace=True)

Nous obtennons au final la liste complète des accidents impliquants un vélo:

In [ ]:
# Finally print result
print(v_df)

            Num_Acc
0      202100000001
9      202100000006
63     202100000038
92     202100000051
120    202100000070
...             ...
97018  202100056346
97041  202100056362
97119  202100056404
97159  202100056424
97298  202100056508

[5851 rows x 1 columns]


##### CONCLUSION
Nous obtennons donc la liste complètes des accidents impliquants des vélos avec leur numéro d'accident.
Nous n'avons plus qu'à explorer les données correspondantes à ces numéros pour pouvoir faire notre *data visualization*.

In [ ]:
c_url: str | None = get_base_url(2021, 'caractéristiques')
if c_url == None:
    exit(0)

"""
    Only use those columns because
    we only want to extract the accident id from the vehicule category
"""
c_col: list[str] = ['Adr', 'Lat', 'Long']

# convert .csv file to pandas dataframe
c_df: pd.DataFrame = pd.read_csv(c_url, sep=';', usecols=c_col, on_bad_lines='skip')

ValueError: Usecols do not match columns, columns expected but not found: ['Adr', 'Lat', 'Long']